# IMDB sentiment analysis using LSTM and GloVe embedding.

Initially we do the similar implementation like a typical LSTM. But, here we use the embedding layer of GloVe embedding later.

# What is GloVe embedding ?

GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space.
GloVe is essentially a log-bilinear model with a weighted least-squares objective. The main intuition underlying the model is the simple observation that ratios of word-word co-occurrence probabilities have the potential for encoding some form of meaning.
Source :https://nlp.stanford.edu/projects/glove/

GloVe helps in understanding better context of words and the meanings attached to them as it works on the principle of co-occourances of the words.

In [24]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
import re

# Data Downloading

Here we simply download the data and put it into a tensorflow data set at the beginning.

In [2]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xzf aclImdb_v1.tar.gz
!ls

--2021-12-07 19:58:45--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  40.3MB/s    in 2.0s    

2021-12-07 19:58:47 (40.3 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]

aclImdb  aclImdb_v1.tar.gz  sample_data


In [3]:
!pip install tensorflow-datasets > /dev/null

In [4]:
import tensorflow_datasets as tfds

In [5]:
(ds_train,ds_test),ds_info = tfds.load(
    name="imdb_reviews",
    split=["train","test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteG8GDXO/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteG8GDXO/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


# Steps for implementation:

1 Data processing: cleaning the data, removing stopwords, splitting into X_test, y_test, X_train, y_train and Finally, Calculating maxlen for encoding

2 Tokenization and encoding: Tokenization and encoding X_train and X_test

3 GloVe embedding: Downloading GloVe and creating the embedding matrix

4 LSTM modelling: LSTM model and Bidirectional LSTM

PS: different explained and detail in comments. When implementing.

# Data Processing.

Here we firstly convert the data into a dataframe since it is easier to deal with in a data frame. Then post that we clean the data and make it comatible for our further analysis.


In [6]:
# we create a dataframe from a tensorflow data object
#we take a higher value than 25000 in take() so that we do not miss any values
ds_train = tfds.as_dataframe(ds_train.take(25100), ds_info)
ds_test = tfds.as_dataframe(ds_test.take(25100), ds_info)

In [7]:
ds_train.head(5)
#we see these b's as data converts to bytes hence we need to decode the bytes and do some basic cleaning to the data set
# its probably because of utf 8, the data is not easy to deal with if it is in the byte format for further processing

,label,text
0,0,"b""This was an absolutely terrible movie. Don't..."
1,0,b'I have been known to fall asleep during film...
2,0,b'Mann photographs the Alberta Rocky Mountains...
3,1,b'This is the kind of film for a snowy Sunday ...
4,1,"b'As others have mentioned, all the women that..."


In [8]:
# As we can see their are some weird characters in between, lets do the very basic cleaning
def basic_clean(txt):
  txt = txt.decode("utf-8") #to remove b's from the beginning of the text and make it string
  txt = re.compile("[.;:!\'?,\"()\[\]]").sub("", txt.lower()) #remove punctuations change and text to lower case
  txt = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)").sub(" ", txt.lower()) #remove links since we want to do textual analysis
  return txt
ds_train['text'] =  ds_train['text'].apply(basic_clean) #apply the above function to train part of data set
ds_test['text'] =  ds_test['text'].apply(basic_clean) #apply the above function to test part of data set
ds_train.head(5)

,label,text
0,0,this was an absolutely terrible movie dont be ...
1,0,i have been known to fall asleep during films ...
2,0,mann photographs the alberta rocky mountains i...
3,1,this is the kind of film for a snowy sunday af...
4,1,as others have mentioned all the women that go...


In [9]:
pd.set_option('display.max_colwidth', 1000) #to view more of data in the data frame
ds_train.head(10) #check the application of the function

,label,text
0,0,this was an absolutely terrible movie dont be lured in by christopher walken or michael ironside both are great actors but this must simply be their worst role in history even their great acting could not redeem this movies ridiculous storyline this movie is an early nineties us propaganda piece the most pathetic scenes were those when the columbian rebels were making their cases for revolutions maria conchita alonso appeared phony and her pseudo love affair with walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning i am disappointed that there are movies like this ruining actors like christopher walkens good name i could barely sit through it
1,0,i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the sette and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow and boring things seemed to happen but with no explanation of what was causing them or why i admit i may have missed part of the film but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else i cant recommend this film at all
2,0,mann photographs the alberta rocky mountains in a superb fashion and jimmy stewart and walter brennan give enjoyable performances as they always seem to do but come on hollywood a mountie telling the people of dawson city yukon to elect themselves a marshal yes a marshal and to enforce the law themselves then gunfighters battling it out on the streets for control of the town nothing even remotely resembling that happened on the canadian side of the border during the klondike gold rush mr mann and company appear to have mistaken dawson city for deadwood the canadian north for the american wild west canadian viewers be prepared for a reefer madness type of enjoyable howl with this ludicrous plot or to shake your head in disgust
3,1,this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm chair and mellow for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no rapids to cross no dangerous waters just a warm and witty paddle through new york life at its best a family film in every sense and one that deserves the praise it received
4,1,as others have mentioned all the women that go nude in this film are mostly absolutely gorgeous the plot very ably shows the hypocrisy of the female libido when men are around they want to be pursued but when no men are around they become the pursuers of a 14 year old boy and the boy becomes a man really fast we should all be so lucky at this age he then gets up the courage to pursue his true love
5,1,this is a film which should be seen by anybody interested in effected by or suffering from an eating disorder it is an amazingly accurate and sensitive portrayal of bulimia in a teenage girl its causes and its symptoms the girl is played by one of the most brilliant young actresses working in cinema today alison lohman who was later so spectacular in where the truth lies i would recommend that this film be shown in all schools as you will never see a better on this subject alison lohman is absolutely outstanding and one marvels at her ability to convey the anguish of a girl suffering from this compulsive disorder if barometers tell us the air pressure alison lohman tells us the emotional pressure with the same degree of accuracy her emotional range is so precise each scene could be measured microscopically for its gradations of trauma on a scale of rising hysteria and desperation which reaches unbearable intensity mare winningham is the perfect choice to play her mother and does so...
6,0,okay you have penelope keith as miss herringbone tweed bbe backbone

In [10]:
#split the data into X_train, y_train, X_test, y_test
X_train = ds_train['text']
y_train = ds_train['label']
X_test = ds_test['text']
y_test = ds_test['label']

In [11]:
#remove stopwords from test and train. We remove stopwords because since these words occour a lot and don't add much meaning to the sentences
nltk.download('stopwords')
english_stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
X_test = X_test.replace({'[^A-Za-z]': ' '}, regex = True) #remove non alphabet for textual analysis
X_test = X_test.apply(lambda text: [w for w in text.split() if w not in english_stops])  #remove stop words

In [13]:
X_train = X_train.replace({'[^A-Za-z]': ' '}, regex = True)     #remove non alphabet for textual analysis
X_train = X_train.apply(lambda text: [w for w in text.split() if w not in english_stops])  # remove stop words

In [14]:
#function to max_len for texts_to_sequences
def get_maxlen():
    review_len = []
    for txt in X_train:
        review_len.append(len(txt))
    return int(np.ceil(np.mean(review_len)))
    #we use the mean value of the lengths of the texts in the data
    #mean is a bit higher than the meidan as median is 90 and mean is 122 hence i went with mean

# Tokenization and Encoding Data

In [15]:
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(X_train) #we fit it on train data it creates tokens using train data as its corpus
X_train = token.texts_to_sequences(X_train) #it creates sequences using the tokens from the above fit_on_texts
X_test = token.texts_to_sequences(X_test) #it uses the same tokens from the train part to create its sequences and removes the words from the sequence which are unique to test data

max_length = get_maxlen() #it is the maximum length sentence for the analysis

X_train = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post') #encoded x train, this makes the sequence of a fixed length of max len and truncates the sequence if its longer than max len and if it shorter it provides the sequence padding to make sequence of uniform max len
X_test = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post') #encoded x test, it does the same to test data

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', X_train, '\n')
print('Encoded X Test\n', X_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[  312   284     1 ...     0     0     0]
 [  451   674  2211 ...     0     0     0]
 [ 4198  5932 26692 ...     0     0     0]
 ...
 [  750    90    53 ...  4587   475  1171]
 [  116  1333   138 ...     0     0     0]
 [   18 16630   276 ...   294     7  4731]] 

Encoded X Test
 [[   25    23  3887 ...     0     0     0]
 [36686   571   662 ...     3 19996   603]
 [  504     1  1547 ...   111 20541   176]
 ...
 [28021     1   103 ...     0     0     0]
 [  492   745  8648 ...     0     0     0]
 [   20    23  2263 ...     0     0     0]] 

Maximum review length:  122


In [16]:
total_words #total number of unique words in train

90275

# GloVe Embedding

In [17]:
#downloading GloVe 
!wget http://nlp.stanford.edu/data/glove.6B.zip  
!unzip glove.6B.zip

--2021-12-07 20:00:57--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-12-07 20:00:57--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-12-07 20:00:57--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [18]:
#opening the glove file and putting in the dictionary of word as key and value as the vector values
import numpy as np
dictwv = {} 
with open('glove.6B.300d.txt', "r") as file:     
    for line in file:         
       tokens = line.split()      
       word = tokens[0]         
       vector = np.array(tokens[1:], dtype=np.float32) 
       if vector.shape[0] == 300:    
           dictwv[word] = vector 
       else:      
           print("There was an issue with " + word) 
# let's check the vocabulary size 
print("Dictionary Size: ", len(dictwv)) 

Dictionary Size:  400000


In [19]:
#dimensions of the embed matrix
embed_dim = 300  
embed_matrix = np.zeros((total_words, embed_dim)) 

In [20]:
word_index = token.word_index 

In [21]:
#find words and its respective embed vector value using tokens and giving the respective value for forming the matrix
for word, i in word_index.items():
    if i < total_words:
        embd_vec = dictwv.get(word) #get the value for the word
        if embd_vec is not None:
            embed_matrix[i] = embd_vec #forming the matrix

In [22]:
embed_matrix #embedding matrix which will be used to initialize the embedding layer

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.138     , -0.12203   ,  0.0054643 , ...,  0.19934   ,
         0.057473  , -0.023767  ],
       [-0.030351  , -0.17344999, -0.097576  , ...,  0.26174   ,
         0.083567  , -0.19064   ],
       ...,
       [ 0.61458999, -0.45030999,  0.51901001, ...,  0.37097001,
         0.40219   , -0.5431    ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

# LSTM Model with GloVe

In [ ]:
from keras.initializers import Constant
LSTM_1 = 32 #lstm cells
from keras import backend as K
K.clear_session() #clear tensor

model = Sequential() #instantiate the model
model.add(Embedding(total_words, 300, embeddings_initializer = Constant(embed_matrix), input_length = max_length, trainable = False)) #we give the input and output dimensions of the embedding layer as the size of vocabulary as input dimension and 300 as the output dimension as its the size of matrix 300d further we initialize the weights of the layer as the embedding embedding matrix which is defined by GloVe and further we set the layer as trainable = false because we do not want the GloVe weights to change  
model.add(LSTM(LSTM_1, dropout=0.3, recurrent_dropout=0.2)) #lstm, we use dropout and recurrent dropout since they have been found successful in textual analysis in lstms
model.add(Dense(1, activation='sigmoid')) #we use dense =1 and activation = sigmoid since binary output
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])  #we use binary_crossentropy because of the type of output metric as accuracy since the data set is balanced in negative and positive reviews and we settle on rms prop as optimizer beacuse it gave the best result compared to others

print(model.summary()) #summary

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 122, 300)          27081600  
                                                                 
 lstm (LSTM)                 (None, 32)                42624     
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 27,124,257
Trainable params: 42,657
Non-trainable params: 27,081,600
_________________________________________________________________
None


In [ ]:
history = model.fit(X_train, y_train, batch_size = 128, epochs = 30) #model training

Epoch 1/30
196/196 [==============================] - 130s 643ms/step - loss: 0.5867 - accuracy: 0.6882
Epoch 2/30
196/196 [==============================] - 125s 638ms/step - loss: 0.4791 - accuracy: 0.7858
Epoch 3/30
196/196 [==============================] - 125s 640ms/step - loss: 0.4367 - accuracy: 0.8030
Epoch 4/30
196/196 [==============================] - 125s 639ms/step - loss: 0.4148 - accuracy: 0.8203
Epoch 5/30
196/196 [==============================] - 124s 634ms/step - loss: 0.3940 - accuracy: 0.8316
Epoch 6/30
196/196 [==============================] - 126s 643ms/step - loss: 0.3767 - accuracy: 0.8384
Epoch 7/30
196/196 [==============================] - 125s 637ms/step - loss: 0.3643 - accuracy: 0.8418
Epoch 8/30
196/196 [==============================] - 125s 636ms/step - loss: 0.3438 - accuracy: 0.8537
Epoch 9/30
196/196 [==============================] - 124s 634ms/step - loss: 0.3398 - accuracy: 0.8571
Epoch 10/30
196/196 [==============================] - 125s 638m

In [ ]:
score, accu = model.evaluate(X_test, y_test,
                            batch_size=128) #testing the model


196/196 [==============================] - 11s 55ms/step - loss: 0.3510 - accuracy: 0.8653


In [ ]:
print('Test accuracy:', accu)

Test accuracy: 0.8653200268745422


In [ ]:
#Test accuracy 83.1 for 100 lstm 50d no dropout 20 epoch adam
#Test accuracy 83.6 for 100 lstm 100d no dropout 20 epoch adam
#Test accuracy 84.8 for 32 lstm 100d dropout 0.2 20 epoch adam
#Test accuracy 85.11 for 32 lstm 100d dropout 0.2 20 epoch rmsprop
#Test accuracy 85 for 32 lstm 100d dropout 0.2 20 epoch nadam
#Test accuracy 85.5  32 lstm 300d dropout 0.2 20 epoch nadam
#Test accuracy 85.5 32 lstm 300d dropout 0.2 20 epoch rmsprop
#Test accuracy 86 32 lstm 300d dropout 0.3 20 epoch rmsprop
#Test accuracy 86.3 32 lstm 300d dropout 0.3 30 epoch rmsprop
#Test accuracy 86.5 32 lstm 300d dropout 0.3, recurrent 0.2 30 epoch rmsprop


# Bidirectional LSTM

Bidirectional models try to understand the context of the sentence from left to right and right to left and later concatenate it. Which makes a worthwhile approach to try in this scenario.

In [56]:
from keras.initializers import Constant
LSTM_1 = 32 #lstm cells
from keras import backend as K
K.clear_session() #clear tensor

model = Sequential() #instantiate the model
model.add(Embedding(total_words, 300, embeddings_initializer = Constant(embed_matrix), input_length = max_length, trainable = False)) #we give the input and output dimensions of the embedding layer as the size of vocabulary as input dimension and 300 as the output dimension as its the size of matrix 300d further we initialize the weights of the layer as the embedding embedding matrix which is defined by GloVe and further we set the layer as trainable = false because we do not want the GloVe weights to change  
model.add(Bidirectional(LSTM(LSTM_1, return_sequences=True))) #Bidirectional Layer
model.add(Bidirectional(LSTM(LSTM_1))) #bidirectional later
model.add(Dense(1, activation='sigmoid')) #since binary output we use dense 1 and sigmoid as activation
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])  #we use binary_crossentropy because of the type of output metric as accuracy since the data set is balanced in negative and positive reviews and we settle on rms prop as optimizer beacuse it gave the best result compared to others

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 122, 300)          27082500  
                                                                 
 bidirectional (Bidirectiona  (None, 122, 64)          85248     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               24832     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 27,192,645
Trainable params: 110,145
Non-trainable params: 27,082,500
_________________________________________________________________
None


In [57]:
history = model.fit(X_train, y_train, batch_size = 128, epochs = 6) #model training, bidirectional models overfit quite quickly hence we keep less epochs

Epoch 1/6
196/196 [==============================] - 30s 112ms/step - loss: 0.4921 - accuracy: 0.7655
Epoch 2/6
196/196 [==============================] - 22s 111ms/step - loss: 0.3893 - accuracy: 0.8286
Epoch 3/6
196/196 [==============================] - 22s 111ms/step - loss: 0.3530 - accuracy: 0.8469
Epoch 4/6
196/196 [==============================] - 22s 112ms/step - loss: 0.3296 - accuracy: 0.8606
Epoch 5/6
196/196 [==============================] - 22s 112ms/step - loss: 0.3051 - accuracy: 0.8743
Epoch 6/6
196/196 [==============================] - 22s 111ms/step - loss: 0.2862 - accuracy: 0.8812


In [58]:
score, accu = model.evaluate(X_test, y_test,
                            batch_size=128) #testing the model

196/196 [==============================] - 12s 52ms/step - loss: 0.3368 - accuracy: 0.8523


In [60]:
print('Test accuracy:', accu)

Test accuracy: 0.8522800207138062


# Conclusion

With GloVe embeddings the standard LSTM (86.5) performs a bit better compared to without glove embedding (85.3)
Bidirectional LSTMs are both at similar level (85.2 and 85.6) in this case in terms of accuracy. 

Bidirectional LSTM in this case is much more efficient compared to standard LSTM with GloVe because it requires a 24 less epochs to reach the similar accuracy as the standard LSTM.